In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

In [2]:
nb_vertical = 9
nb_horizontal = 6


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((nb_horizontal*nb_vertical,3), np.float32)
objp[:,:2] = np.mgrid[0:nb_vertical,0:nb_horizontal].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints_l = [] # 3d point in real world space
imgpoints_l = [] # 2d points in image plane.

subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)


images = sorted(glob.glob('Stereo_calibration_images/left*.png'))

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, (nb_vertical, nb_horizontal), cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints_l.append(objp)
        cv2.cornerSubPix(gray,corners,(11, 11),(-1,-1),subpix_criteria)
        imgpoints_l.append(corners)
        
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nb_vertical,nb_horizontal), corners,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

cv2.destroyAllWindows()


In [3]:
objp = np.zeros((nb_horizontal*nb_vertical,3), np.float32)
objp[:,:2] = np.mgrid[0:nb_vertical,0:nb_horizontal].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints_r = [] # 3d point in real world space
imgpoints_r = [] # 2d points in image plane.


images = sorted(glob.glob('Stereo_calibration_images/right*.png'))

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nb_vertical, nb_horizontal), cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints_r.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),subpix_criteria)
        imgpoints_r.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nb_vertical,nb_horizontal), corners,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

cv2.destroyAllWindows()

In [4]:
ret, mtx_left, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints_l, imgpoints_l, gray.shape[::-1], None, None)
img = cv2.imread('Stereo_calibration_images/left-0000.png')
h,  w = img.shape[:2]
ret, mtx_right, dist_2, rvecs, tvecs = cv2.calibrateCamera(objpoints_r, imgpoints_r, gray.shape[::-1], None, None)
print(mtx_left)
print(dist)
print(mtx_right)
print(dist_2)

[[703.59270499   0.         621.96204913]
 [  0.         703.547427   372.69127092]
 [  0.           0.           1.        ]]
[[-3.28857966e-01  1.40839430e-01 -2.03002374e-04  1.09785667e-04
  -3.09299570e-02]]
[[700.01614988   0.         647.63446636]
 [  0.         700.21423829 372.89905976]
 [  0.           0.           1.        ]]
[[-3.27205041e-01  1.40667566e-01 -1.11023941e-04  5.00502395e-04
  -3.24744770e-02]]


In [5]:
img_shape = gray.shape[::-1]

0.23572728777371055


In [9]:
#Stereo calibrate
stereocalibration_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)
stereocalibration_flags = (cv2.CALIB_FIX_K3 + cv2.CALIB_FIX_K4 + cv2.CALIB_FIX_K5 + cv2.CALIB_FIX_ASPECT_RATIO + cv2.CALIB_ZERO_TANGENT_DIST + cv2.CALIB_SAME_FOCAL_LENGTH + cv2.CALIB_RATIONAL_MODEL) 

retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = cv2.stereoCalibrate(objpoints_l, imgpoints_l, imgpoints_r, mtx_left, dist, mtx_right, dist_2, gray.shape[::-1],flags = stereocalibration_flags, criteria = stereocalibration_criteria)

In [10]:
print(retval)
print(cameraMatrix1)
print(distCoeffs1)
print(cameraMatrix2)
print(distCoeffs2)
print(R)
print(T)
print(E)
print(F)

0.3077694781947018
[[722.19099627   0.         632.32322214]
 [  0.         722.26963485 371.60953929]
 [  0.           0.           1.        ]]
[[-0.33910572  0.16526134  0.          0.          0.          0.
   0.          0.06360177  0.          0.          0.          0.
   0.          0.        ]]
[[722.19099627   0.         648.20872308]
 [  0.         722.26963485 374.01901843]
 [  0.           0.           1.        ]]
[[-0.34530767  0.17408995  0.          0.          0.          0.
   0.          0.06880116  0.          0.          0.          0.
   0.          0.        ]]
[[ 0.99996277 -0.00565091  0.00652116]
 [ 0.0056628   0.99998234 -0.00180546]
 [-0.00651084  0.00184232  0.99997711]]
[[-3.57473805]
 [-0.00774091]
 [ 0.05689866]]
[[-2.71805676e-04 -5.69119145e-02 -7.63800345e-03]
 [ 3.36219762e-02  6.26426898e-03  3.57502726e+00]
 [-1.25023940e-02 -3.57471865e+00  6.50451031e-03]]
[[-1.97434490e-07 -4.13352347e-05  1.14786214e-02]
 [ 2.44197070e-05  4.54925544e-06  1.8

In [11]:
P1 = np.zeros(shape=(3,4)) 
P2 = np.zeros(shape=(3,4))

R1 = np.zeros(shape=(3,3))
R2 = np.zeros(shape=(3,3))

R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(cameraMatrix1,distCoeffs1,cameraMatrix2,distCoeffs2,gray.shape[::-1], R, T, alpha=0)

print("R1: \n")
print(R1)
print("R2: \n")
print(R2)
print("P1: \n")
print(P1)
print("P2: \n")
print(P2)
print("Q: \n")
print(Q)
print("ROI 1: \n")
print(roi1)
print("ROI 2: \n")
print(roi2)

R1: 

[[ 9.99949662e-01 -3.51437290e-03 -9.39804547e-03]
 [ 3.50601979e-03  9.99993444e-01 -9.05140419e-04]
 [ 9.40116486e-03  8.72145123e-04  9.99955428e-01]]
R2: 

[[ 9.99871007e-01  2.16516848e-03 -1.59148217e-02]
 [-2.15088818e-03  9.99997269e-01  9.14357406e-04]
 [ 1.59167580e-02 -8.80008458e-04  9.99872933e-01]]
P1: 

[[720.63825687   0.         682.02740479   0.        ]
 [  0.         720.63825687 410.10417175   0.        ]
 [  0.           0.           1.           0.        ]]
P2: 

[[ 7.20638257e+02  0.00000000e+00  6.82027405e+02 -2.57642534e+03]
 [ 0.00000000e+00  7.20638257e+02  4.10104172e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
Q: 

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -6.82027405e+02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -4.10104172e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  7.20638257e+02]
 [ 0.00000000e+00  0.00000000e+00  2.79704693e-01 -0.00000000e+00]]
ROI 1: 

(0, 0, 1280, 720)

In [13]:
undistortion_map_left, rectification_map_left = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1,img_shape,cv2.CV_32FC1) 
undistortion_map_right, rectification_map_right = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2,img_shape,cv2.CV_32FC1) 


In [16]:
img1 = cv2.imread('Stereo_calibration_images/left-0020.png')
img2 = cv2.imread('Stereo_calibration_images/right-0020.png')

img_rect1 = cv2.remap(img1, undistortion_map_left, rectification_map_left, cv2.INTER_LINEAR)
img_rect2 = cv2.remap(img2, undistortion_map_right, rectification_map_right, cv2.INTER_LINEAR)


# draw the images side by side
total_size = (max(img_rect1.shape[0], img_rect2.shape[0]),
                  img_rect1.shape[1] + img_rect2.shape[1], 3)
img = np.zeros(total_size, dtype=np.uint8)
img[:img_rect1.shape[0], :img_rect1.shape[1]] = img_rect1
img[:img_rect2.shape[0], img_rect1.shape[1]:] = img_rect2

# draw horizontal lines every 25 px accross the side by side image
for i in range(20, img.shape[0]-50, 35):
    cv2.line(img, (0, i), (img.shape[1], i), (255, 0, 0))
    cv2.imshow('imgRectified', img)
    cv2.waitKey(500)

cv2.imwrite("test.png", img);
cv2.destroyAllWindows()
